<a href="https://colab.research.google.com/github/ShirinLiu/ML_Course/blob/master/06_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import argparse

######################
# Optimization Flags #
######################

learning_rate = 0.001 # initial learning rate
seed = 111

##################
# Training Flags #
##################
batch_size = 128 # Batch size for training
num_epoch = 10 # Number of training iterations

###############
# Model Flags #
###############
hidden_size = 128 # Number of neurons for RNN hodden layer

# Reset the graph set the random numbers to be the same using "seed"
tf.reset_default_graph()
tf.set_random_seed(seed)
np.random.seed(seed)

# Divide 28x28 images to rows of data to feed to RNN as sequantial information
step_size = 28
input_size = 28
output_size = 10

# Input tensors
X = tf.placeholder(tf.float32, [None, step_size, input_size])
y = tf.placeholder(tf.int32, [None])

# creat BasicRnn
# 'outputs' is a tensor of shape [batch_size, max_time, 256]
# 'state' is a N-tuple where N is the number of LSTMCells containing a
# tf.contrib.rnn.LSTMStateTuple for each cell
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# Forward pass and loss calcualtion
logits = tf.layers.dense(state, output_size)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Prediction
# 找計算出的結果和實際結果的差異
prediction = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

# input data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

# Process MNIST
X_test = mnist.test.images # X_test shape: [num_test, 28*28]
X_test = X_test.reshape([-1, step_size, input_size])
y_test = mnist.test.labels

# initialize the variables
init = tf.global_variables_initializer()

# Empty list for tracking
loss_train_list = []
acc_train_list = []

# train the model
with tf.Session() as sess:
    sess.run(init)
    n_batches = mnist.train.num_examples // batch_size
    for epoch in range(num_epoch):
        for batch in range(n_batches):
            X_train, y_train = mnist.train.next_batch(batch_size)
            X_train = X_train.reshape([-1, step_size, input_size])
            sess.run(optimizer, feed_dict={X: X_train, y: y_train})
        loss_train, acc_train = sess.run(
            [loss, accuracy], feed_dict={X: X_train, y: y_train})
        loss_train_list.append(loss_train)
        acc_train_list.append(acc_train)
        print('Epoch: {}, Train Loss: {:.3f}, Train Acc: {:.3f}'.format(
            epoch + 1, loss_train, acc_train))
    loss_test, acc_test = sess.run(
        [loss, accuracy], feed_dict={X: X_test, y: y_test})
    print('Test Loss: {:.3f}, Test Acc: {:.3f}'.format(loss_test, acc_test))

W0731 05:22:50.951490 140157560960896 deprecation.py:323] From <ipython-input-17-dc6c3a05d58e>:39: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0731 05:22:51.317257 140157560960896 deprecation.py:323] From <ipython-input-17-dc6c3a05d58e>:43: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 1, Train Loss: 0.279, Train Acc: 0.898
Epoch: 2, Train Loss: 0.125, Train Acc: 0.969
Epoch: 3, Train Loss: 0.166, Train Acc: 0.977
Epoch: 4, Train Loss: 0.217, Train Acc: 0.922
Epoch: 5, Train Loss: 0.094, Train Acc: 0.945
Epoch: 6, Train Loss: 0.134, Train Acc: 0.977
Epoch: 7, Train Loss: 0.157, Train Acc: 0.977
Epoch: 8, Train Loss: 0.112, Train Acc: 0.945
Epoch: 9, Train Loss: 0.060, Train Acc: 0.984
Epoch: 10, Train Loss: 0.047, Train Acc: 0.992
Test Loss: 0.107, Test Acc: 0.970
